## Задание 3

Написать скрипт - парсер, ИСПОЛЬЗУЯ SELENIUM.

Обязательно должен иметь пагинацию
Желательно авторизацию

Парсим что угодно, новостной сайт, интернет-магазин, без разницы.

Параметров сохраняет не менее 4-х.

Записывает всё в csv файл.

Далее он же будет завязан на следующем задании.

In [41]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time



### Авторизация

In [77]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)
driver.maximize_window()


In [66]:
# Define the URL
url = "https://henderson.ru/"
driver.get(url)
driver.implicitly_wait(15)



In [67]:
import time
time.sleep(2) 

In [68]:
WebDriverWait(driver, 10).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)


login_link = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((
        By.XPATH, 
        '//a[normalize-space()="Войти"]' 
    ))
)
login_link.click()

In [69]:
email_radio_div = driver.find_element(By.XPATH, '//input[@type="radio" and @value="email"]/following-sibling::div')
driver.execute_script("arguments[0].click();", email_radio_div)


In [70]:

email_input = wait.until(EC.visibility_of_element_located((
    By.XPATH, 
    '//div[@id="auth-email"]//input[@name="email"]'
)))
email_input.clear()
email_input.send_keys("artemsa1505@mail.ru")


password = wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@type="password"]')))
password.clear()
password.send_keys("df99ghjbnm")

In [71]:
button = wait.until(EC.visibility_of_element_located((By.XPATH, '//button[normalize-space()="Войти"]' )))
button.click()

In [72]:
time.sleep(5) 

In [73]:
buttons = driver.find_elements(By.CLASS_NAME, "js-send-sms")
buttons[1].click()
print(f"Найдено кнопок: {len(buttons)}")

code = int(input("Введите код из СМС:"))

Найдено кнопок: 2


ValueError: invalid literal for int() with base 10: ''

### Парсинг

In [51]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)
driver.maximize_window()

In [55]:
url_blazers = "https://henderson.ru/catalog/blazers/"
driver.get(url_blazers)
wait = WebDriverWait(driver, 5)

In [ ]:
data = []
page = 1
i = 0

while True:
    try:
        element = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "list-product"))
        )

        products = driver.find_elements(By.CLASS_NAME, "list-product__item")

        for product in products:
            title = product.get_attribute("data-product-name").strip()
            price = product.get_attribute("data-product-unitsaleprice")
            category = product.get_attribute("data-product-category")
            link = product.find_element(By.CSS_SELECTOR, ".card-product__name a").get_attribute("href")
            img_url = product.find_element(By.CSS_SELECTOR, ".card-product__wrap-img img").get_attribute("src")

            i +=1

            data.append({
                "id_item": i,
                "title": title,
                "price": price,
                "category": category,
                "link": link,
                "img_url": img_url
            })

    except TimeoutException as _ex:
        print(_ex)
        break

    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination-next")  # Находим ссылку "Следующая"
        next_page_url = next_button.get_attribute("href") 

        driver.get(next_page_url)

        page += 1
        time.sleep(5)

    except:
        print(f"Число страниц: {page}")
        
        break
    


✅ Достигли последней страницы! Число страниц: 3


In [57]:
for item in data:
    print(item)

{'id_item': 1, 'title': 'JT-0378-R LNAVY', 'price': '42999', 'category': '["Одежда","Пиджаки повседневные"]', 'link': 'https://henderson.ru/product/jt-0378-r-lnavy/', 'img_url': 'https://cdnstatic.henderson.ru/images/catalog/product/JT-0378-R/JT-0378-R-LNAVY-5-1.jpg'}
{'id_item': 2, 'title': 'JT-0374-R NAVY', 'price': '21999', 'category': '["Одежда","Пиджаки повседневные"]', 'link': 'https://henderson.ru/product/jt-0374-r-navy/', 'img_url': 'https://cdnstatic.henderson.ru/images/catalog/product/JT-0374-R/JT-0374-R-NAVY-5-1.jpg'}
{'id_item': 3, 'title': 'Повседневный пиджак из эластичной ткани, прямой силуэт JT-0369-R KHAKI', 'price': '21999', 'category': '["Одежда","Пиджаки повседневные"]', 'link': 'https://henderson.ru/product/jt-0369-r-khaki/', 'img_url': 'https://cdnstatic.henderson.ru/images/catalog/product/JT-0369-R/JT-0369-R-KHAKI-5-1.jpg'}
{'id_item': 4, 'title': 'Пиджак повседневный из хлопка и льна на мужчин среднего телосложения JT-0381-R BLUE', 'price': '23999', 'category': 

In [ ]:

df = pd.DataFrame(data)
df.to_csv('data.csv')